# Lab 6

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('/content/drive/MyDrive/Academic/ML/lab5/data/winequality-red.csv', delimiter=';')
data.head()


FileNotFoundError: ignored

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
sns.countplot(x="quality", data=data)

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(data.corr(method='pearson'), annot=True, square=True)
plt.show()

In [ ]:
bins = (2, 6.5, 8)
group_names = ['bad', 'good']
data['quality'] = pd.cut(data['quality'], bins = bins, labels = group_names)
data.head()

In [ ]:
data['quality'] = data['quality'].map({'bad' : 0, 'good' : 1})
data.head(10)

In [ ]:
print(data['quality'].value_counts())
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(18, 5))
ax = ax.flatten()
data['quality'].value_counts().plot(x=0, y=1, kind='pie', figsize=(15,5), ax=ax[0])
sns.countplot(data['quality'], ax=ax[1])
plt.show()

In [ ]:
X = data.drop('quality', axis = 1)
y = data['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train_not_scaled = X_train
X_test_not_scaled = X_test


X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()

model = clf.fit(X_train, y_train)
pred_gaussian = clf.predict(X_test)
print(pred_gaussian)

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes, normalize = False,
                          title='Confusion matrix',
                          cmap=plt.cm.Greens):
    plt.figure(figsize = (5, 5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, size = 24)
    plt.colorbar(aspect=4)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, size = 14)
    plt.yticks(tick_marks, classes, size = 14)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    # лейблы для плотов
    for i, j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
             plt.text(j, i, format(cm[i, j], fmt),
             fontsize = 20,
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label', size = 18)
    plt.xlabel('Predicted label', size = 18)


cm = confusion_matrix(y_test, pred_gaussian)
plot_confusion_matrix(cm, classes = ['0 - Bad', '1 - Good'],
                      title = 'Confusion Matrix')

K-Nearest Neighbour Classification

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN

knn = KNN(n_neighbors=5, metric='minkowski', p=2,)
knn.fit(X_train, y_train)
pred_knn = knn.predict(X_test)
Y_compare_knn = pd.DataFrame({'Actual' : y_test, 'Predicted' : pred_knn})
print(Y_compare_knn.head())
print('\nConfussion matrix:')
print(confusion_matrix(y_test, pred_knn))

In [ ]:
cm = confusion_matrix(y_test, pred_knn)
plot_confusion_matrix(cm, classes = ['0 - Bad', '1 - Good'],
                      title = 'Confusion Matrix')

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='lbfgs', random_state=42)
logreg.fit(X_train, y_train)
pred_logreg = logreg.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, pred_logreg)
plot_confusion_matrix(cm, classes = ['0 - Bad', '1 - Good'],
                      title = 'Confusion Matrix')

In [ ]:
from sklearn.model_selection import cross_val_score

#K-fold cross validation
modelNames = ['Logistic Regression', 'K-Nearest Neighbour', 'Naive Bias']
modelClassifiers = [logreg, knn, clf]
models = pd.DataFrame({'modelNames' : modelNames, 'modelClassifiers' : modelClassifiers})
counter=0
score=[]
for i in models['modelClassifiers']:
  accuracy = cross_val_score(i, X_train, y_train, scoring='accuracy', cv=10)
  print('Accuracy of %s Classification model is %.2f' %(models.iloc[counter,0],accuracy.mean()))
  score.append(accuracy.mean())
  counter+=1

In [ ]:
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd
from sklearn.model_selection import cross_val_score

# tp / (tp + fn)
def calculate_recall(clf, X_train, y_train):
    recall_scorer = make_scorer(recall_score)
    recall = cross_val_score(clf, X_train, y_train, scoring=recall_scorer, cv=5)
    return recall.mean()

# tp / (tp + fp)
def calculate_precision(clf, X_train, y_train):
    precision_scorer = make_scorer(precision_score)
    precision = cross_val_score(clf, X_train, y_train, scoring=precision_scorer, cv=5)
    return precision.mean()

def calculate_f1_score(clf, X_train, y_train):
    f1_scorer = make_scorer(f1_score)
    f1 = cross_val_score(clf, X_train, y_train, scoring=f1_scorer, cv=5)
    return f1.mean()

# tn / (tn + fp)
def calculate_specificity(clf, X_train, y_train):
    def specificity_score(y_true, y_pred):
        tn, fp, _, _ = confusion_matrix(y_true, y_pred).ravel()
        specificity = tn / (tn + fp)
        return specificity

    specificity_scorer = make_scorer(specificity_score)
    specificity = cross_val_score(clf, X_train, y_train, scoring=specificity_scorer, cv=5)
    return specificity.mean()

modelNames = ['Logistic Regression', 'K-Nearest Neighbour', 'Naive Bias']
modelClassifiers = [logreg, knn, clf]
models = pd.DataFrame({'modelNames' : modelNames, 'modelClassifiers' : modelClassifiers})
counter = 0
score = []

for i in models['modelClassifiers']:
    # For Recall
    recall = calculate_recall(i, X_train, y_train)
    print('Recall of %s Classification model is %.2f' %(models.iloc[counter,0], recall))
    score.append(recall)

    # For Precision
    precision = calculate_precision(i, X_train, y_train)
    print('Precision of %s Classification model is %.2f' %(models.iloc[counter,0], precision))
    score.append(precision)

    # For F1 Score
    # F1=PPV+TPR2⋅PPV⋅TPR​=2⋅TP+FP+FN2⋅TP​.
    f1 = calculate_f1_score(i, X_train, y_train)
    print('F1 Score of %s Classification model is %.2f' %(models.iloc[counter,0], f1))
    score.append(f1)

    # For Specificity
    specificity = calculate_specificity(i, X_train, y_train)
    print('Specificity of %s Classification model is %.2f' %(models.iloc[counter,0], specificity))
    score.append(specificity)

    print()

    counter += 1


In [ ]:
# Определяем метрики
from sklearn import metrics

y_pred_proba = logreg.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_proba)

#create ROC curve
plt.plot(fpr, tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


In [ ]:
from sklearn.metrics import roc_auc_score

# Получаем вероятности положительного класса для тестового набора данных
y_pred_proba = logreg.predict_proba(X_test)[:,1]

# Вычисляем AUC-ROC
roc_auc = roc_auc_score(y_test, y_pred_proba)

print('AUC-ROC:', roc_auc)


In [ ]:
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, precision_recall_curve, auc
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import cross_val_predict

def plot_pr_curve(y_true, y_scores, model_name):
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    auc_pr = auc(recall, precision)

    plt.figure(figsize=(8, 8))
    plt.plot(recall, precision, label=f'{model_name} (AUC-PR = {auc_pr:.2f})', linewidth=2)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve: {model_name}')
    plt.legend(loc='best')
    plt.show()

modelNames = ['Logistic Regression', 'K-Nearest Neighbour', 'Naive Bias']
modelClassifiers = [logreg, knn, clf]
models = pd.DataFrame({'modelNames': modelNames, 'modelClassifiers': modelClassifiers})
counter = 0
score = []

for i in models['modelClassifiers']:
    # For Precision-Recall Curve and AUC-PR
    y_scores = cross_val_predict(i, X_train, y_train, method='predict_proba', cv=5)[:, 1]
    plot_pr_curve(y_train, y_scores, models.iloc[counter, 0])

    counter += 1
